In [1]:
import cv2
import datetime
import os
import serial
import shutil
import time
import torch

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch.nn.functional as F
import seaborn as sb

from collections import OrderedDict
# from labjack import ljm
from PIL import Image
from torch import nn
from torch import optim
from torchvision import datasets, transforms, models
from typing import Tuple

from skimage.metrics import structural_similarity as compare_ssim

In [2]:
def take_picture(cam_ID: int, width: int, height: int, name_ID: int, crop: bool = False):
    cap = cv2.VideoCapture(cam_ID)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
    path = '/home/michael/Cax_Box_Pics/unsorted'
    if cap.isOpened():
        for i in range(20):
            _,_ = cap.read()
        ret, frame = cap.read()
        if crop:
            crop_img = frame[0:-100, 150:-75]
            cv2.imwrite(
                os.path.join(path , "litterbox_cropped_" + str(name_ID) + ".png"),
                crop_img)
        else:
            cv2.imwrite(
                os.path.join(path , "litterbox_" + str(name_ID) + ".png"),
                frame)
        cap.release()
        cv2.destroyAllWindows()
        return frame

# def toggle_fan(handle, fan_state: int) -> None:
#     ljm.eWriteName(handle, "FIO3", fan_state)

def toggle_fan(fan_state: int) -> None:
    s = serial.Serial('/dev/ttyACM0', 9600)
    if fan_state == 0:
        s.write(b'fan_off')
        s.write(b'\n')
    elif fan_state == 1:
        s.write(b'fan_on')
        s.write(b'\n')
    s.close()

In [3]:
def load_checkpoint(filepath: str):
    """ Reload a saved .pth model."""
    checkpoint = torch.load(filepath)

    if checkpoint['arch'] == 'vgg16':
        model = models.vgg16(pretrained=True)
        for param in model.parameters():
            param.requires_grad = False
    else:
        print("Architecture not recognized.")

    model.class_to_idx = checkpoint['class_to_idx']
    classifier = nn.Sequential(
        OrderedDict([('fc1', nn.Linear(25088, 5000)),
                    ('relu', nn.ReLU()),
                    ('drop', nn.Dropout(p=0.5)),
                    ('fc2', nn.Linear(5000, 102)),
                    ('output', nn.LogSoftmax(dim=1))]))
    model.classifier = classifier
    model.load_state_dict(checkpoint['model_state_dict'])
    return model


def process_image(image_path: str) -> np.array:
    ''' Image preprocessor for inference.

        Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''

    # Process a PIL image for use in a PyTorch model
    pil_image = Image.open(image_path)

    # Resize
    if pil_image.size[0] > pil_image.size[1]:
        pil_image.thumbnail((5000, 256))
    else:
        pil_image.thumbnail((256, 5000))

    # Crop
    left_margin = (pil_image.width-224)/2
    bottom_margin = (pil_image.height-224)/2
    right_margin = left_margin + 224
    top_margin = bottom_margin + 224

    pil_image = pil_image.crop((left_margin, bottom_margin,
                                right_margin, top_margin))

    # Normalize
    np_image = np.array(pil_image)/255
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    np_image = (np_image - mean) / std

    # PyTorch expects the color channel to be the first dimension but it's the
    # third dimension in the PIL image and Numpy array
    # Color channel needs to be first; retain the order of the other two
    # dimensions.
    np_image = np_image.transpose((2, 0, 1))

    return np_image


def predict(image_path, model, topk: int = 3):
    ''' Predict the class of an image using trained deep learning model.'''

    image = process_image(image_path)

    # Convert image to PyTorch tensor first
    
    #CUDA
    # image = torch.from_numpy(image).type(torch.cuda.FloatTensor)
    
    #CPU
    image = torch.from_numpy(image).type(torch.FloatTensor)

    # Returns a new tensor with a dimension of size one inserted
    # at the specified position.
    image = image.unsqueeze(0)
    output = model.forward(image)
    probabilities = torch.exp(output)

    # Probabilities and the indices of those probabilities
    # corresponding to the classes
    top_prob, top_ind = probabilities.topk(topk)

    # Convert to lists
    top_prob = top_prob.detach().type(torch.FloatTensor).numpy().tolist()[0]
    top_ind = top_ind.detach().type(torch.FloatTensor).numpy().tolist()[0]

    # Convert topk_indices to the actual class labels using class_to_idx
    # Invert the dictionary so you get a mapping from index to class.

    idx_to_class = {value: key for key, value in model.class_to_idx.items()}
    top_classes = [idx_to_class[index] for index in top_ind]

    return top_prob, top_classes

def _check_image(image_path: str, inf_model):
    probs, classes = predict(image_path, model)
    print(classes)
    print(probs)
    print('\n')
    return classes[0], float(probs[0])

In [ ]:
%matplotlib inline
path = '/home/michael/Cax_Box_Pics/unsorted'
config_path = '/home/michael/Cax_Box_Pics'
config_path = os.path.join(config_path , "cax_box_config" + ".conf")
model = load_checkpoint('Feb_6_2021_basic_vgg.pth')
# handle = ljm.openS("T7", "USB", "ANY")
toggle_fan(0)

with open(config_path) as f:
    image_counter = [line for line in f]

image_counter = int(image_counter[0])
image_last = take_picture(0, 640, 480, image_counter, crop = True)
gray_last = cv2.cvtColor(image_last, cv2.COLOR_BGR2GRAY)
previous_state = "0"
zero_counter = 1
wait_time = 60

while True:
    try:
        image = take_picture(0, 640, 480, image_counter, crop = True)
        image_path = os.path.join(path , "litterbox_cropped_"
                                  + str(image_counter) + ".png")
        f = open(config_path, "w")
        f.write(str(image_counter + 1))
        f.close()

        box_state_inf, box_state_prob = _check_image(image_path, model)
        if box_state_inf == "0":
            zero_counter += 1
        if box_state_inf == "0" and zero_counter % 20 == 0:
            shutil.copy(image_path, 'inf//' + str(box_state_inf))
        elif box_state_inf != "0":
            shutil.copy(image_path, 'inf//' + str(box_state_inf))
        gray_current = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        score, diff = compare_ssim(gray_last, gray_current, full=True)
        wait_time = 60
        if box_state_prob >= 0.90 and score <= 0.90:
            if box_state_inf == "1" and previous_state == "0":
                toggle_fan(1)
                previous_state == "1"
                wait_time = 300
            elif box_state_inf == "0" and previous_state == "1":
                toggle_fan(0)
                previous_state = "0"
                wait_time = 300
            else:
                toggle_fan(0)
            if box_state_inf == "2":
                previous_state = "0"
                toggle_fan(0)
        gray_last = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    except Exception as e:
        print(e)
    image_counter += 1
    time.sleep(wait_time)
    if image_counter % 10 == 0:
        print(datetime.datetime.now())

['1', '0', '2']
[0.9996969699859619, 0.00030302649247460067, 3.711676105355721e-11]


['1', '0', '2']
[0.9999997615814209, 1.8628271902798588e-07, 5.2940853469737176e-14]


['1', '0', '2']
[0.9127306342124939, 0.08726724982261658, 2.059290181932738e-06]


['1', '0', '2']
[0.9999474287033081, 5.262402191874571e-05, 1.9544847684738187e-10]


2021-02-05 17:18:46.184521
['1', '0', '2']
[0.9987271428108215, 0.001272592693567276, 2.1017081053287257e-07]


['1', '0', '2']
[0.99793541431427, 0.002050795592367649, 1.383900143991923e-05]


['1', '0', '2']
[0.9999730587005615, 2.6923924451693892e-05, 5.707662609211184e-08]


['1', '0', '2']
[0.999971866607666, 2.811905687849503e-05, 3.07722076986483e-12]


['1', '0', '2']
[0.9990266561508179, 0.0009733175975270569, 4.016816740337603e-11]


['1', '0', '2']
[0.9925305247306824, 0.00746775371953845, 1.793023102436564e-06]


['0', '1', '2']
[0.5403709411621094, 0.45962876081466675, 2.4497100525877613e-07]


['1', '0', '2']
[0.9997952580451965, 0.0002

['1', '0', '2']
[0.9999998807907104, 1.055602538713174e-07, 1.0204384004453004e-09]


['1', '0', '2']
[0.9999964237213135, 3.60009039468423e-06, 7.998052886648566e-09]


['1', '0', '2']
[0.9999979734420776, 2.0577413124556188e-06, 3.5028584477458935e-09]


['1', '0', '2']
[0.9999957084655762, 4.296872702980181e-06, 1.9095623693399233e-10]


['1', '0', '2']
[0.9999996423721313, 3.147077052290115e-07, 1.0082120027732344e-08]


['1', '0', '2']
[0.9999982118606567, 1.728972165437881e-06, 2.807059584775118e-10]


['1', '0', '2']
[0.9999991655349731, 8.335700272255053e-07, 2.3894528577983465e-09]


['1', '0', '2']
[1.0, 1.251847603356282e-08, 6.690363324110549e-13]


['1', '0', '2']
[0.9999998807907104, 6.805869645631901e-08, 3.524879832461636e-10]


2021-02-05 19:13:11.749112
['1', '0', '2']
[0.9999998807907104, 7.668116097647726e-08, 7.511909427826424e-10]


['1', '0', '2']
[0.9999995231628418, 4.93782522426045e-07, 9.031582426377582e-11]


['1', '0', '2']
[0.9999876022338867, 1.2449217138

['1', '0', '2']
[0.9999998807907104, 8.757597669273309e-08, 1.0335946321049505e-08]


['1', '0', '2']
[0.9999992847442627, 7.305076792363252e-07, 6.307671052141473e-11]


2021-02-05 20:56:57.418172
['1', '0', '2']
[0.9999492168426514, 5.033290290157311e-05, 4.378886728773068e-07]


['1', '0', '2']
[1.0, 5.913989298278466e-08, 8.50922354800332e-09]


['1', '2', '0']
[0.9983397722244263, 0.0016412490513175726, 1.8948605429613963e-05]


['1', '0', '2']
[0.9986996650695801, 0.0013003385392948985, 1.1653600751060367e-08]


['1', '0', '2']
[0.9989266991615295, 0.0010649212636053562, 8.295916813949589e-06]


['1', '0', '2']
[0.999134361743927, 0.0008639908628538251, 1.6376590110667166e-06]


['1', '0', '2']
[0.9984392523765564, 0.001560673350468278, 8.652448713064587e-08]


['1', '0', '2']
[0.9992493987083435, 0.0007503150845877826, 2.8246992656022485e-07]


['1', '0', '2']
[0.9998973608016968, 0.00010267813922837377, 6.629296417370867e-11]


['1', '0', '2']
[0.9950880408287048, 0.00491200573

['1', '0', '2']
[0.9988503456115723, 0.0011495717335492373, 1.2604640176050452e-07]


['1', '0', '2']
[0.9998561143875122, 0.00014392103184945881, 4.5008059501583375e-09]


['1', '0', '2']
[0.9999788999557495, 2.1090047084726393e-05, 5.394859003970964e-10]


['1', '0', '2']
[0.9999829530715942, 1.7105647202697583e-05, 2.8274008911921555e-08]


['1', '0', '2']
[0.9995762705802917, 0.0004235550295561552, 8.383698002489837e-08]


['1', '0', '2']
[0.9949030876159668, 0.005096678622066975, 2.0110026355268928e-07]


['1', '0', '2']
[0.9999172687530518, 7.959065260365605e-05, 3.0446849450527225e-06]


['1', '0', '2']
[0.989540159702301, 0.010439793579280376, 2.0088233213755302e-05]


2021-02-05 22:51:48.805092
['1', '0', '2']
[0.9999430179595947, 5.689205136150122e-05, 1.4910649781540997e-07]


['1', '0', '2']
[0.9989621639251709, 0.0010137333301827312, 2.4039052732405253e-05]


['1', '0', '2']
[0.9923698902130127, 0.007623827084898949, 6.221341664058855e-06]


['1', '0', '2']
[0.999951243400

['1', '0', '2']
[0.999220609664917, 0.000772200059145689, 7.183161415014183e-06]


['1', '0', '2']
[0.8425490856170654, 0.15741421282291412, 3.6776855267817155e-05]


